In [ ]:
#To install
#https://phoenixnap.com/kb/install-spark-on-windows-10
#https://towardsdatascience.com/how-to-use-pyspark-on-your-computer-9c7180075617

In [1]:
import findspark
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
import pandas as pd
findspark.init()
spark = SparkSession.builder.getOrCreate()

In [2]:
path = "C:/Users/garci/Documents/Wizeline/all_data.csv"
df = spark.read.csv(path, header=True, inferSchema= True)

In [3]:
df.select("estado").distinct().toPandas()

,estado
0,QUINTANA ROO
1,NUEVO LEÓN
2,SINALOA
3,TABASCO
4,BAJA CALIFORNIA
5,TLAXCALA
6,COAHUILA DE ZARAGOZA
7,None
8,"ESQ. SUR 125"""
9,CHIAPAS


In [4]:
#bad_states = ['None', ' ESQ. SUR 125"', ' COL. EDUARDO GUERRA', "estado"]
df.filter(col("estado") == "None").show(1)
df.filter(col("estado") == ' ESQ. SUR 125"').show(1)
df.filter(col("estado") == ' COL. EDUARDO GUERRA').show(1)
df.filter(col("estado") == "estado").show(1)

+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|producto|presentacion|marca|categoria|catalogo|precio|fechaRegistro|cadenaComercial|giro|nombreComercial|direccion|estado|municipio|latitud|longitud|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+

+------------------+------------+-----+---------+--------+------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+----------------+--------------------+---------+
|          producto|presentacion|marca|categoria|catalogo|precio|       fechaRegistro|     cadenaComercial|                giro|nombreComercial|           direccion| 

In [5]:
path = "C:/Users/garci/Documents/Wizeline/all_data.csv"
df = spark.read.option("quote", "\"").option("escape", "\"").csv(path, header=True, inferSchema= True)
df = df.filter((col("estado") != "None") & (col("estado") != "estado")) 

In [6]:
df.select("estado").distinct().toPandas() 

,estado
0,QUINTANA ROO
1,NUEVO LEÓN
2,SINALOA
3,TABASCO
4,BAJA CALIFORNIA
5,TLAXCALA
6,COAHUILA DE ZARAGOZA
7,CHIAPAS
8,VERACRUZ DE IGNACIO DE LA LLAVE
9,SONORA


## 1. How many commercial chains are monitored, and therefore, included in this database?

In [7]:
df.agg(countDistinct("cadenaComercial").alias("Number of commercial chains")).collect()[0] 

Row(Number of commercial chains=704)

## 2. What are the top 10 monitored products by State?

In [8]:
df = df.withColumn("unique product", concat(col("producto"), lit(" "), col("presentacion"), lit(" "), col("marca")))
prod_counts = df.groupby("estado","unique product").count()

window = (Window
        .partitionBy(prod_counts["estado"])
        .orderBy(prod_counts["count"].desc()))

res = (prod_counts.select(col("*"), row_number().over(window).alias("rank"))
.filter(col("rank") <= 10)
.orderBy(col("estado"), col("count").desc()))

res_pd = res.drop("count").groupby("estado").pivot("rank").agg(first("unique product")).orderBy(col("estado")).toPandas()

In [9]:
res_pd

,estado,1,2,3,4,5,6,7,8,9,10
0,AGUASCALIENTES,TORTILLA DE MAIZ 1 KG. GRANEL S/M,LECHE ULTRAPASTEURIZADA CAJA 1 LT. ENTERA LALA...,LECHE PASTEURIZADA CAJA 1 LT. LALA. ENTERA,PAN BLANCO BOLILLO PIEZA S/M,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,ACEITE BOTELLA 1 LT. MIXTO 1-2-3,LIMPIADOR LIQUIDO P/PISO BOTELLA 1 LT. MULTIUS...,JABON DE PASTA BARRA 400 GR. CON ENVOLTURA ZOT...,REFRESCO BOTELLA 2.5 LT. NO RETORNABLE COCA COLA,LECHE CONDENSADA LATA 397 GR. LA LECHERA.
1,BAJA CALIFORNIA,TORTILLA DE MAIZ 1 KG. GRANEL S/M,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,ACEITE BOTELLA 1 LT. MIXTO 1-2-3,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,REFRESCO LATA 355 ML. COCA COLA,ATUN LATA 140 GR. EN ACEITE. LOMO DOLORES,ATUN LATA 140 GR. EN AGUA. LOMO DOLORES,PAN DE CAJA PAQUETE 210 GR. TOSTADO. CLÁSICO B...,DETERGENTE P/ROPA BOLSA 1 KG. POLVO FOCA,SALSA PICANTE BOTELLA 150 GR. CLASICA BUFALO
2,BAJA CALIFORNIA SUR,TORTILLA DE MAIZ 1 KG. GRANEL S/M,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,PASTELILLOS PAQUETE CON 1 GANSITO MARINELA,REFRESCO LATA 355 ML. COCA COLA,HARINA DE MAIZ PAQUETE 1 KG. MASECA,MOLE ROJO EN PASTA VASO 235 GR. DONA MARIA,PAN DE CAJA PAQUETE 210 GR. TOSTADO. CLÁSICO B...,CREMA VASO 200 ML. LALA,PASTELILLOS PAQUETE CON 2 PINGUINOS MARINELA
3,CAMPECHE,TORTILLA DE MAIZ 1 KG. GRANEL S/M,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,ELOTE LATA 220 GR. EN GRANO HERDEZ,MOLE ROJO EN PASTA VASO 235 GR. DONA MARIA,LECHE ULTRAPASTEURIZADA PARCIALMENTE DESCREMAD...,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,PAN DE CAJA PAQUETE GRANDE 680 GR. REBANADO BL...,CREMA VASO 200 ML. LALA,SAN RAFAEL. REAL 1 KG. GRANEL. JAMON. DE PIERN...,LECHE ULTRAPASTEURIZADA CAJA 1 LT. SEMIDESCREM...
4,CHIAPAS,TORTILLA DE MAIZ 1 KG. GRANEL S/M,CHILES EN LATA LATA 220 GR. JALAPENOS RAJAS LA...,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,LECHE ULTRAPASTEURIZADA CAJA 1 LT. SEMIDESCREM...,LECHE ULTRAPASTEURIZADA PARCIALMENTE DESCREMAD...,REFRESCO LATA 355 ML. COCA COLA,LECHE ULTRAPASTEURIZADA CAJA 1 LT. ENTERA LALA...,PURE DE TOMATE CAJA 1 KG. CONDIMENTADO DEL FUERTE,SALSA PICANTE BOTELLA 150 GR. CLASICA BUFALO,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX
5,CHIHUAHUA,TORTILLA DE MAIZ 1 KG. GRANEL S/M,ACEITE BOTELLA 1 LT. MIXTO 1-2-3,CHILES EN LATA LATA 220 GR. JALAPENOS RAJAS LA...,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,JABON DE PASTA BARRA 400 GR. CON ENVOLTURA ZOT...,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,SHAMPOO BOTELLA 800 ML. ACEITE HERBAL CAPRICE ...,CHILES EN LATA LATA 380 GR. JALAPENOS RAJAS LA...,REFRESCO LATA 355 ML. COCA COLA,ACEITE BOTELLA 946 ML. SOYA NUTRIOLI
6,COAHUILA DE ZARAGOZA,TORTILLA DE MAIZ 1 KG. GRANEL S/M,LECHE ULTRAPASTEURIZADA CAJA 1 LT. ENTERA LALA...,LECHE PASTEURIZADA CAJA 1 LT. LALA. ENTERA,PAN BLANCO BOLILLO PIEZA S/M,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,MOLE ROJO EN PASTA VASO 235 GR. DONA MARIA,ACEITE BOTELLA 946 ML. SOYA NUTRIOLI,HARINA DE MAIZ PAQUETE 1 KG. MASECA,CHILES EN LATA LATA 220 GR. JALAPENOS RAJAS LA...
7,COLIMA,TORTILLA DE MAIZ 1 KG. GRANEL S/M,LECHE ULTRAPASTEURIZADA CAJA 1 LT. ENTERA LALA...,PAN BLANCO BOLILLO PIEZA S/M,LECHE PASTEURIZADA CAJA 1 LT. LALA. ENTERA,LECHE ULTRAPASTEURIZADA PARCIALMENTE DESCREMAD...,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,BLANQUEADOR BOTELLA 950 ML. EL RENDIDOR CLORALEX,MOLE ROJO EN PASTA VASO 235 GR. DONA MARIA,MAYONESA FRASCO 390 GR. CON JUGO DE LIMONES MC...,PURE DE TOMATE CAJA 1 KG. CONDIMENTADO DEL FUERTE
8,DISTRITO FEDERAL,LECHE ULTRAPASTEURIZADA PARCIALMENTE DESCREMAD...,TORTILLA DE MAIZ 1 KG. GRANEL S/M,CREMA VASO 200 ML. ALPURA,PAN BLANCO BOLILLO PIEZA S/M,LECHE CONDENSADA LATA 397 GR. LA LECHERA.,YOGHURT VASO 150 GR. FRESA ALPURA,CREMA VASO 200 ML. LALA,LECHE ULTRAPASTEURIZADA PARCIALMENTE DESCREMAD...,LECHE ULTRAPASTEURIZADA PARCIALMENTE DESCREMAD...,CREMA BOTE 450 ML. ALPURA
9,DURANGO,TORTILLA DE MAIZ 1 KG. GRANEL S/M,ACEITE BOTELLA 1 LT. MIXTO 

## 3. Which is the commercial chain with the highest number of monitored products?

In [15]:
df.groupby("cadenaComercial").agg(countDistinct("unique product").alias("count")).orderBy(col("count").desc()).show(1) 

+---------------+-----+
|cadenaComercial|count|
+---------------+-----+
|       WAL-MART| 7371|
+---------------+-----+
only showing top 1 row



## 4. Use the data to find an interesting fact.

We can see that in every state the 1 KG. tortilla is the top 1 product except on Distrito Federal in which it appears on second, this is to be expected as maize is the staple food on Mexico. On second place we tend to see various kinds of milk.  

These statistic count the appearance of the product on different stores, we can conclude that they are widespread but this does not tell us how many of these are being sold.

## 5. What are the lessons learned from this exercise?

We must ensure the data quality of the database before performing analysis. Failing to do this might invalidate an analysis and we might not catch this error before its too late.

## 6. Can you identify other ways to approach this problem? Explain.

If I had more time we would need to do analysis on each of the columns first to try and detect entry errors.  
One problem I identified is that there are city names that are entered incorrectly perhaps due to an extra white space.


In [19]:
df.groupby("estado", "municipio").count().select("estado", "municipio").orderBy("estado", "municipio").show(2, truncate = False) 

+--------------+----------------------------------------+
|estado        |municipio                               |
+--------------+----------------------------------------+
|AGUASCALIENTES|AGUASCALIENTES                          |
|AGUASCALIENTES|AGUASCALIENTES                          |
+--------------+----------------------------------------+
only showing top 2 rows

